<b> Computing Sharpe Ratio on existing shares allocation </b>

<pre>
Symbol  Shares  Acquired
AAPL    420     2014-07-07
GOOG    17      2014-06-26
TSLA    51      2015-03-03
TWTR    1396    2014-07-02
UNH     1464    2014-07-07
</pre>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import util

In [ ]:
#stocks_dir = '/Volumes/Photos/stocks/'
stocks_dir = './'

symbols = ['UNH', 'AAPL', 'GOOG', 'TSLA', 'TWTR']

stocks = {}

for s in symbols:
    file_name = stocks_dir + s + '.csv'
    df = pd.read_csv(file_name, index_col='Date', parse_dates=True, usecols=['Date', 'Adj. Close'])
    stocks[s] = df


sp500 = pd.read_csv('SP500_yahoo.csv', index_col='Date', parse_dates=True, na_values=['.'])
sp500 = sp500[['Adj Close']]
sp500.columns = ['Adj. Close']

symbols.extend(['SP500'])
stocks['SP500']  = sp500


In [ ]:
shares = {'UNH': 1464, 'AAPL': 420, 'GOOG': 17, 'TSLA': 51, 'TWTR': 1396, 'SP500': 0}

for sym, df in stocks.items():
    #df['daily_ret'] = df['Adj. Close'].pct_change(1)
    df['log_ret'] = np.log(df['Adj. Close']/df['Adj. Close'].shift(1))
    df['shares'] = shares[sym]
    df['alloc'] = df['shares'] * df['Adj. Close']
    df.drop(columns=['shares', 'Adj. Close'], inplace=True)
    df.columns = [ sym + ' ' + c for c in df.columns]

In [ ]:
d1, d2 = '2014-04-01', '2018-03-15'
df = pd.concat(stocks.values(), axis=1).loc[d1:d2].copy()

In [ ]:
df['alloc'] = sum([df[sym + ' alloc'] for sym in symbols])

for sym in symbols:
    df[sym + ' weight'] = df[sym + ' alloc'] / df['alloc']

In [ ]:
df.loc['2014-07-02':].head()

In [ ]:
#d1, d2 = '2014-07-02', '2018-01-01'
d1, d2 = '2017-01-01', '2018-01-01'

print("Real Return: {}".format(df.loc[d1:d2].iloc[-1]['alloc'] / df.loc[d1:d2].iloc[0]['alloc']))

df2 = df.loc[d1:d2]

# Expected Return
print('Historic Portfolio Return')
log_ret = df2[[sym + ' log_ret' for sym in symbols]]
log_ret.columns = range(len(log_ret.columns))
#print(log_ret.head())

weights = df2[[sym + ' weight' for sym in symbols]]
weights.columns = range(len(weights.columns))
#print(weights.head())

exp_ret = (log_ret * weights).sum(axis=1).mean() * 252
print(exp_ret + 1)
print('\n')


# Expected Volatility
print('Expected Volatility')
exp_vol = np.sqrt((log_ret * weights).sum(axis=1).var() * 252)
print(exp_vol)
print('\n')

# Sharpe Ratio
SR = exp_ret/exp_vol
print('Sharpe Ratio')
print(SR)

In [ ]:
def sharpe(df, d1, d2):
    
    df2 = df.loc[d1:d2]
    
    # Expected Return
    log_ret = df2[[sym + ' log_ret' for sym in symbols]]
    log_ret.columns = range(len(log_ret.columns))
    weights = df2[[sym + ' weight' for sym in symbols]]
    weights.columns = range(len(weights.columns))
    exp_ret = (log_ret * weights).sum(axis=1).mean() * 252

    # Expected Volatility
    exp_vol = np.sqrt((log_ret * weights).sum(axis=1).var() * 252)

    # Sharpe Ratio
    SR = exp_ret/exp_vol
    return SR

In [ ]:
sharpe(df, d1, d2)

In [ ]:
start_date, end_date, months = '2014-01-01', '2018-01-01', 12
it = util.generate_periods(start_date, end_date, months, months_overlap=0)

for d1, d2 in it:
    print(sharpe(df, d1, d2))

In [ ]:
start_date, end_date, months = '2014-01-01', '2018-01-01', 12
it = util.generate_periods(start_date, end_date, months, months_overlap=6)

for d1, d2 in it:
    print(sharpe(df, d1, d2))

In [ ]:
start_date, end_date, months = '2014-01-01', '2018-01-01', 3
it = util.generate_periods(start_date, end_date, months, months_overlap=1)

for d1, d2 in it:
    print(sharpe(df, d1, d2))